# MS Preprocessing
!To get dataset for this task, first line classifier model has to be available and used to generate line labels on the reports in seantis_kisim.csv (corresponds to inference on the "all" split in the line-label/line-label_clean_dataset)!

Necessary steps before this:
1. notebooks/00_preprocessing_old_project.ipynb
2. notebooks/01_classifying_text_lines.ipynb
3. scripts/line-label/inference.py --model_name "trained-line-classifier" --split all

The data/preprocessed/midatams/seantis_kisim.csv file was created by the original project (00_preprocessing_old_project.ipynb).
This file contains the longest report per rid splitted into lines. Their approach:

1. Extract the longest diagnosis per rid (most lines) from the csv and if the rid had a manually line labelled text, they used this instead.
2. Results in dataset consisting of text lines per row with a label for the line.

Further processing:

3. Label this dataset with clf 1. Merge diagnoses.csv and line labelled dataset by rid. Clean the labels. Correct SPMS and PPMS labels that are wrong.
4. Get a list of eligible rid (rids with text that have at least one dm line).
5. Df1: concatenate all dm lines per eligible rid for the text.
6. Df2: concatenate all text per eligible rid.

In [1]:
import pandas as pd
import torch
import sys
import os
sys.path.append(os.getcwd()+"/../..")

from src import paths
from src.utils import ms_label2id, line_label_id2label

from datasets import DatasetDict, Dataset

from sklearn.model_selection import train_test_split

In [2]:
# Line Labelled dataset Token Finetuned
data = torch.load(paths.RESULTS_PATH/"line-label/line-label_medbert-512_token_all.pt")

data_list = []
for obs in data:
    _df = pd.DataFrame(obs["text"], columns=["text"])
    _df["class2"] = obs["preds"]
    _df["rid"] = obs["rid"]
    data_list.append(_df)

data_token_df = pd.concat(data_list)

data_token_df = data_token_df[["rid", "text", "class2"]]


# Make directory if it doesn't exist
os.makedirs(paths.DATA_PATH_PREPROCESSED/"ms-diag", exist_ok=True)

data_token_df.to_csv(paths.DATA_PATH_PREPROCESSED/"ms-diag/line-label_medbert-512_token_finetuned_all.csv", index=False)

In [3]:
# Line Labelled dataset Line Finetuned
data_line = torch.load(paths.RESULTS_PATH/"line-label/line-label_medbert-512_class_all.pt")
data_line.pop("last_hidden_state")

data_line_df = pd.DataFrame(data_line)
data_line_df.rename(columns={"preds": "class2"}, inplace=True)
data_line_df.drop(columns=["labels"], inplace=True)
data_line_df = data_line_df[["rid", "index_within_rid", "text", "class2"]]

# Remap values in class2 using ms_id2label
data_line_df["class2"] = data_line_df["class2"].map(line_label_id2label)

# Make directory if it doesn't exist
os.makedirs(paths.DATA_PATH_PREPROCESSED/"ms-diag", exist_ok=True)

data_line_df.to_csv(paths.DATA_PATH_PREPROCESSED/"ms-diag/line-label_medbert-512_class_finetuned_all.csv", index=False)

In [4]:
# The line labelled will have more lines because no line truncation
print("Length Line Data: ", len(data_line_df))
print("Length Token Data: ", len(data_token_df))

Length Line Data:  5669
Length Token Data:  4395


In [5]:
data_line_df[data_line_df["rid"] == "016B6D16-2BBA-4C05-A8E4-30F761C95813"]

,rid,index_within_rid,text,class2
0,016B6D16-2BBA-4C05-A8E4-30F761C95813,0,Diagnosen allgemein,head
1,016B6D16-2BBA-4C05-A8E4-30F761C95813,1,INDENT Schubförmige Multiple Sklerose EM 03.06...,dm
2,016B6D16-2BBA-4C05-A8E4-30F761C95813,2,"INDENT 03/2021: klinisch nicht aktiv, radiolog...",his_sym_cu
3,016B6D16-2BBA-4C05-A8E4-30F761C95813,3,INDENT 06/2019 Schub mit Hypästhesie der Beine...,his_sym_cu
4,016B6D16-2BBA-4C05-A8E4-30F761C95813,4,INDENT MRI WS 06/2019: kontrastmittelaufnehmen...,mr
5,016B6D16-2BBA-4C05-A8E4-30F761C95813,5,INDENT cMRI 07/2019: mehrere entzündlich Läsio...,mr
6,016B6D16-2BBA-4C05-A8E4-30F761C95813,6,INDENT LP vom 09.07.2019: Pleozytose mit aktiv...,labr_labo
7,016B6D16-2BBA-4C05-A8E4-30F761C95813,7,INDENT Vaskulitisscreening und infektiologisch...,labr_labo
8,016B6D16-2BBA-4C05-A8E4-30F761C95813,8,"INDENT Migräne ohne Aura, EM ca. 2002",medo_unk_do_so
9,016B6D16-2BBA-4C05-A8E4-30F761C95813,9,"INDENT Depressive Störung, ED 2007",medo_unk_do_so


In [6]:
data_token_df[data_token_df["rid"] == "016B6D16-2BBA-4C05-A8E4-30F761C95813"]

,rid,text,class2
0,016B6D16-2BBA-4C05-A8E4-30F761C95813,Diagnosen allgemein,head
1,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT Schubförmige Multiple Sklerose EM 03.06...,dm
2,016B6D16-2BBA-4C05-A8E4-30F761C95813,"INDENT 03/2021: klinisch nicht aktiv, radiolog...",his_sym_cu
3,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT 06/2019 Schub mit Hypästhesie der Beine...,his_sym_cu
4,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT MRI WS 06/2019: kontrastmittelaufnehmen...,mr
5,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT cMRI 07/2019: mehrere entzündlich Läsio...,mr
6,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT LP vom 09.07.2019: Pleozytose mit aktiv...,labr_labo
7,016B6D16-2BBA-4C05-A8E4-30F761C95813,INDENT Vaskulitisscreening und infektiologisch...,labr_labo
8,016B6D16-2BBA-4C05-A8E4-30F761C95813,"INDENT Migräne ohne Aura, EM ca. 2002",medo_unk_do_so
9,016B6D16-2BBA-4C05-A8E4-30F761C95813,"INDENT Depressive Störung, ED 2007",medo_unk_do_so


## Preparing CLF1 Dataset for MS Diag

- I decide to go with line classifier, even though it is a bit worse, but no truncation issues for longer reports
- The steps are:

1. Merging Datasets on rid (one row is one line in text)
2. Cleaning up labels (only using confirmed diganoses, rewriting German labels, correcting wrong labels for small count classes)
3. Construction of "no_ms" class: all lines labelled as something other than "dm" are "no_ms".
4. Construct 2 datasets:
    - df1: contains only single lines per row.
    - df2: contains all text per rid. If at least one line was "dm" then label was set to the MS Type for this rid. If not "no_ms". More "no_ms" by creating texts, that don't contain the "dm" line. (So there might be multiple examples from same rid, one time with dm line, one time without.)

In [7]:
# Line labelled dataset from classifier 1
df_text = pd.read_csv(os.path.join(paths.DATA_PATH_PREPROCESSED, "ms-diag", "line-label_medbert-512_class_finetuned_all.csv"))[["rid", "text", "class2", "index_within_rid"]]
df_labels = pd.read_csv(os.path.join(paths.DATA_PATH_SEANTIS, "diagnoses.csv"))

# In old approach they only used confirmed diagnosis
df_labels = df_labels[df_labels["diagnosis_reliability"] == "confirmed"]
df_labels = df_labels[["research_id", "disease"]].rename(columns={"disease": "labels", "research_id": "rid"})

In [8]:
# Merge with diagnoses.csv
df_merged = pd.merge(df_text, df_labels, on="rid", how="inner")

In [9]:
# English Labels
english_labels = set(["relapsing_remitting_multiple_sclerosis", "secondary_progressive_multiple_sclerosis", "primary_progressive_multiple_sclerosis"])
other_labels = set(df_merged["labels"].unique()) - english_labels

In [10]:
other_labels

{'Multiple Sklerose',
 'Multiple Sklerose a.e. primär progredient',
 'Multiple Sklerose mit a.e. primär-progredientem Verlauf',
 'Multiple Sklerose mit am ehesten progredientem Verlauf',
 'Progressive multifokale Leukencephalopathie (PML)',
 'Schubförmig remittierende Multiple Sklerose ',
 'Schubförmig remittierende Multiple Sklerose (RRMS)',
 'Schubförmige Multiple Sklerose',
 'St.n. symptomatischer Epilepsie mit einfach fokalen und komplex fokalen Anfällen'}

In [11]:
# Check non english labels
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

for rid, rid_data in df_merged.groupby("rid"):
    if rid_data.labels.isin(other_labels).any():
        print(rid_data.labels.unique())
        print(rid_data["text"].str.cat(sep = " "))

['Multiple Sklerose a.e. primär progredient']
1. Multiple Sklerose a.e. primär progredient, EM 2011, ED 03/2015, EDSS 3.5 INDENT klinisch: nicht aktiv, radiologisch: nicht aktiv, Progression: nein (nach Lublin et al. 2013) INDENT aktuell: im langfristigen Verlauf langsame Besserung der spastisch-ataktischen Gangstörung  Verlauf: INDENT 1997 Episode mit aufsteigenden Sensibilitätsstörungen der Beine. Abklärungen erfolgten in der Neurologie USZ (LP, MRI, evozierte Potentiale), eine Diagnose wurde jedoch nicht gestellt  INDENT 1997-2011 Anamnestisch keine neurologische Symptomatik eruierbar INDENT 2011-2016 langsam progrediente, rechtsbetonte spastisch-ataktische Gangstörung, distal betonte Pallhypästhesie und eingeschränktes Lageempfinden der unteren Extremität Diagnostik: Bildgebung INDENT 06.02.2015 MRI Wirbelsäule: Multietagere multiple T2 hyperintense Läsionen cervicothorakal, V.a. Gd-Aufnahme auf Niveau BWK 5 - BWK 7 INDENT 18.02.2015 MRI Gehirn: Multiple T2w- und FLAIRw-hyperintens

In [12]:
# Remap non english labels if possible
map_dict = {
    "Multiple Sklerose a.e. primär progredient": "primary_progressive_multiple_sclerosis",
    "Multiple Sklerose mit a.e. primär-progredientem Verlauf": "primary_progressive_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose (RRMS)": "relapsing_remitting_multiple_sclerosis",
}
df_merged = df_merged.replace(map_dict)

In [13]:
# Remove all non english labels
df_merged = df_merged[df_merged["labels"].isin(english_labels)]

In [14]:
# Because mapping was done manually, check if label matches text for classes with low counts like SPMS
for rid, rid_data in df_merged[df_merged["labels"] == "secondary_progressive_multiple_sclerosis"].groupby("rid"):
    print(rid)
    print(rid_data["text"].str.cat(sep = "\n"))
    print("\n")

2A9F4832-B09D-470A-B05F-519854310DBB
Schubförmige Multiple Sklerose (ED 1990), EDSS 6.5, V.a. sekundär-progredienten Verlauf
INDENT aktuell: seit Ende 03/2018 aufgrund schwerer Belastungssituation am Arbeitsplatz zunehmende schmerzhafte Krämpfe aller Extremitäten rechts- und beinbetont, Zunahme der Beinschwäche, zusätzlich Gedächtnis- und Konzentrationsstörungen, am Eintrittstag bei allgemeinem Schwächegefühl nicht mehr aufstehen können
INDENT anamnestisch: Gehstrecke zuvor 150 m mit Gehhilfe, vermehrte Sturzanfälligkeit, Harninkontinenz, leichte Fatigue-Symptomatik
INDENT klinisch: rechts- und beinbetonte spastische Tetraparese, Gang- und Standataxie, Sensibilitätsminderung der rechten Extremität, neurogene Blasenfunktionsstörung 
INDENT Verlauf:
INDENT EM: Müdigkeit schon als Kind
INDENT ED 1990: beide Beine plötzlich gelähmt
INDENT 05/2009: deutliche Verschlechterung der vorbestehenden Gang- und Standunsicherheit durch starken Schwankschwindel
INDENT 2010-2011: Gehhilfe
INDENT 2012:

In [15]:
spms_wrong = ["2A9F4832-B09D-470A-B05F-519854310DBB",
              "39D432B0-902B-49D9-B727-12EDC053B09E",
              "AF834D8D-F7DB-4B22-BB01-29F10EE6A828",
              "B886879A-5109-46FD-A2B0-9DCA2DA733F8",
              "C0784569-1E15-4FBE-A4B2-F9473975D199"
                ]
df_merged[df_merged["labels"] == "secondary_progressive_multiple_sclerosis"].rid.unique().shape
# Because of this exclusion we end up with less training examples than in their original approach
df_merged[(df_merged["labels"] == "relapsing_remitting_multiple_sclerosis") & df_merged["text"].str.lower().str.contains("spms|sekundär")]

,rid,text,class2,index_within_rid,labels
157,0FA0E814-EFF9-4888-8410-C27659833E25,"1. Sekundär progrediente schubförmige Multiple Sklerose, EM ca. 1993, ED 08/2004 EDSS 4.0",dm,0,relapsing_remitting_multiple_sclerosis
158,0FA0E814-EFF9-4888-8410-C27659833E25,"klinisch nicht aktiv, radiologisch aktiv, sekundäre Progredienz seit 2013 (nach Lublin et al. 2013)",his_sym_cu,1,relapsing_remitting_multiple_sclerosis
359,189A2297-8248-470D-8378-66B37264856C,DD sekundär progredienter Verlauf mit aufgesetzten Schüben,his_sym_cu,1,relapsing_remitting_multiple_sclerosis
703,328AFD0B-0ADD-4DA8-B4FB-4CA08D63781D,"INDENT St.n. zweimaligen, primär fokalen, sekundär generalisierten epileptischen Anfällen 04/2014, a.e. symptomatisch i.R. der Grunderkrankung; seither unter antiepileptischer Therapie anfallsfrei",his_sym_cu,12,relapsing_remitting_multiple_sclerosis
1883,88F16735-6B66-448A-B88E-75DFBA12A6A2,"Schubförmige Multiple Sklerose mit V. a. sekundär chronisch progredienten Verlauf seit ca. 2016, EM 1997, ED 2003, EDSS 6.0",dm,0,relapsing_remitting_multiple_sclerosis
1920,88F16735-6B66-448A-B88E-75DFBA12A6A2,INDENT 07/2016 - 11/2016: Tysabri 300mg i.v. alle 4 Wochen (Umstellung aufgrund MRI-Aktivität und V.a. sekundäre Progredienz,medms,37,relapsing_remitting_multiple_sclerosis
2369,A3141CEA-1444-4DDC-955F-12149A7BBFB9,39-jährige Patientin mit sekundärer Sterilität seit 06/2010,medo_unk_do_so,0,relapsing_remitting_multiple_sclerosis


In [16]:
# Drop entries with wrong label
df_merged = df_merged[~df_merged["rid"].isin(spms_wrong)]

# Remap entry 157 to secondary_progressive_multiple_sclerosis
df_merged.loc[157, "labels"] = "secondary_progressive_multiple_sclerosis"

In [17]:
# Check primary_progressive_multiple_sclerosis
for rid, rid_data in df_merged[df_merged["labels"] == "primary_progressive_multiple_sclerosis"].groupby("rid"):
    print(rid)
    print(rid_data["text"].str.cat(sep = "\n"))
    print("\n")

0A7AF4C2-ADD6-4B72-9869-4D190928F8C3
v.v Primar progrediente multiple Sklerose


11A4453B-76B6-4558-8126-EEFEA6935C91
1. Multiple Sklerose a.e. primär progredient, EM 2011, ED 03/2015, EDSS 3.5
INDENT klinisch: nicht aktiv, radiologisch: nicht aktiv, Progression: nein (nach Lublin et al. 2013)
INDENT aktuell: im langfristigen Verlauf langsame Besserung der spastisch-ataktischen Gangstörung 
Verlauf:
INDENT 1997 Episode mit aufsteigenden Sensibilitätsstörungen der Beine. Abklärungen erfolgten in der Neurologie USZ (LP, MRI, evozierte Potentiale), eine Diagnose wurde jedoch nicht gestellt 
INDENT 1997-2011 Anamnestisch keine neurologische Symptomatik eruierbar
INDENT 2011-2016 langsam progrediente, rechtsbetonte spastisch-ataktische Gangstörung, distal betonte Pallhypästhesie und eingeschränktes Lageempfinden der unteren Extremität
Diagnostik:
Bildgebung
INDENT 06.02.2015 MRI Wirbelsäule: Multietagere multiple T2 hyperintense Läsionen cervicothorakal, V.a. Gd-Aufnahme auf Niveau BWK 5 - 

In [18]:
# Rids with diagnosis
rids_dm = set(df_merged[df_merged["class2"] == "dm"]["rid"].unique())

# Rids without diagnosis
rids_no_dm = set(df_merged["rid"].unique()) - rids_dm

In [19]:
# Set labels of rids without diagnosis to no_ms
df_merged.loc[df_merged["rid"].isin(rids_no_dm), "labels"] = "no_ms"

In [20]:
# Take all non dm lines and set label to no dm
df1_no_dm = df_merged[df_merged["class2"] != "dm"]
df1_no_dm.loc[:, "labels"] = "no_ms"

# For the rids in rids_dm, extract all lines with class2 == dm
df1_dm = df_merged[df_merged["class2"] == "dm"].groupby("rid").agg({"text": "\n".join, "labels": "first", "index_within_rid": "first"}).reset_index()

# Concat both dataframes
df1 = pd.concat([df1_no_dm, df1_dm]).drop(columns=["class2"])

In [21]:
# For the rids in rids_dm, extract all lines and keep original label
df2 = df_merged.groupby("rid").agg({"text": "\n".join, "labels": "first"}).reset_index()

In [22]:
# df3 will be df2 but put first text line last
df3 = df2.copy()
df3["text"] = df3["text"].apply(lambda x: x.split("\n"))
df3["text"] = df3["text"].apply(lambda x: x[1:] + [x[0]])
df3["text"] = df3["text"].apply(lambda x: "\n".join(x))

In [23]:
df2.labels.value_counts(), df3.labels.value_counts()

(labels
 relapsing_remitting_multiple_sclerosis      131
 no_ms                                        40
 primary_progressive_multiple_sclerosis       13
 secondary_progressive_multiple_sclerosis      9
 Name: count, dtype: int64,
 labels
 relapsing_remitting_multiple_sclerosis      131
 no_ms                                        40
 primary_progressive_multiple_sclerosis       13
 secondary_progressive_multiple_sclerosis      9
 Name: count, dtype: int64)

In [24]:
len(df1.rid.unique()), len(df2.rid.unique()), len(df3.rid.unique())

(193, 193, 193)

In [25]:
# Train Val Test split
df2train, df2test = train_test_split(df2, test_size=0.3, random_state=42, stratify=df2["labels"])
df2train, df2val = train_test_split(df2train, test_size=0.1, random_state=42, stratify=df2train["labels"])

train_rids = set(df2train["rid"].unique())
val_rids = set(df2val["rid"].unique())
test_rids = set(df2test["rid"].unique())

df3train = df3[df3["rid"].isin(train_rids)]
df3val = df3[df3["rid"].isin(val_rids)]
df3test = df3[df3["rid"].isin(test_rids)]

df1train = df1[df1["rid"].isin(train_rids)]
df1val = df1[df1["rid"].isin(val_rids)]
df1test = df1[df1["rid"].isin(test_rids)]

In [26]:
len(df1test.rid.unique()), len(df2test.rid.unique()), len(df3test.rid.unique()) 

(58, 58, 58)

In [27]:
# Create HuggingFace Dataset
def create_hf_dataset(train:pd.DataFrame, val:pd.DataFrame, test:pd.DataFrame):
    """Create HuggingFace Dataset from train, val and test dataframes. Remaps labels to ids and drops unnecessary columns.
    
    Args:
        train (pd.DataFrame): Training dataframe
        val (pd.DataFrame): Validation dataframe
        test (pd.DataFrame): Test dataframe
        
        Returns:
            DatasetDict: HuggingFace DatasetDict
            
    """
    dataset = DatasetDict({
        "train": Dataset.from_pandas(train),
        "val": Dataset.from_pandas(val),
        "test": Dataset.from_pandas(test),
    })

    # Map the labels to ids
    dataset = dataset.map(lambda e: {"labels": [ms_label2id[l] for l in e["labels"]]}, batched=True)

    # Drop __index_level_0__ column
    dataset = dataset.remove_columns(["__index_level_0__"])

    return dataset

dataset1 = create_hf_dataset(df1train, df1val, df1test)
dataset2 = create_hf_dataset(df2train, df2val, df2test)
dataset3 = create_hf_dataset(df3train, df3val, df3test)

# Save the dataset
dataset1.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "ms-diag/ms_diag_line"))
dataset2.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "ms-diag/ms_diag_all"))
dataset3.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "ms-diag/ms_diag_all_first_line_last"))

Map:   0%|          | 0/2243 [00:00<?, ? examples/s]

Map:   0%|          | 0/247 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/58 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/121 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/58 [00:00<?, ? examples/s]

# Summary

In [28]:
# Label distribution
print("Label distribution all:")
print(df2.labels.value_counts(), "\n\n")

print("Label distribution all_first_line_last:")
print(df3.labels.value_counts(), "\n\n")

print("Label distribution line:")
print(df1.labels.value_counts(), "\n\n")

Label distribution all:
labels
relapsing_remitting_multiple_sclerosis      131
no_ms                                        40
primary_progressive_multiple_sclerosis       13
secondary_progressive_multiple_sclerosis      9
Name: count, dtype: int64 


Label distribution all_first_line_last:
labels
relapsing_remitting_multiple_sclerosis      131
no_ms                                        40
primary_progressive_multiple_sclerosis       13
secondary_progressive_multiple_sclerosis      9
Name: count, dtype: int64 


Label distribution line:
labels
no_ms                                       3387
relapsing_remitting_multiple_sclerosis       131
primary_progressive_multiple_sclerosis        13
secondary_progressive_multiple_sclerosis       9
Name: count, dtype: int64 




## Pipeline Approach

To get a fair comparison, I need to retrain the line-classifier with the test rids excluded

In [29]:
# Save test rids in a file
with open(os.path.join(paths.DATA_PATH_PREPROCESSED, "ms-diag/test_rids.txt"), "w") as f:
    f.write("\n".join(test_rids))

In [30]:
# Load line labelled dataset
from src.utils import load_line_label_data
from datasets import concatenate_datasets

line_labels = load_line_label_data()
line_labels_all = concatenate_datasets([line_labels["train"], line_labels["val"], line_labels["test"]])

# Create test set from line labelled dataset by using the test rids
line_labels_test = line_labels_all.filter(lambda e: e["rid"] in test_rids)

# Remove test rids from line labelled dataset
line_labels_all = line_labels_all.filter(lambda e: e["rid"] not in test_rids)

# Cast labels column to ClassLabel and split into train and test
line_labels_all = line_labels_all.class_encode_column("labels").train_test_split(test_size=0.1, shuffle=True, seed=42, stratify_by_column="labels")

# Assign correct splits
line_labels_all["val"] = line_labels_all["test"]
line_labels_all["test"] = line_labels_test
line_labels_all["all"] = line_labels["all"]

# Save the dataset
line_labels_all.save_to_disk(os.path.join(paths.DATA_PATH_PREPROCESSED, "line-label/line-label_clean_dataset_pipeline"))

Saving the dataset (0/1 shards):   0%|          | 0/1133 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/283 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/126 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5669 [00:00<?, ? examples/s]